In [1]:
!pip install langchain==0.1.14 langchain-google-genai==0.0.11 google-generativeai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.4/137.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.7/598.7 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/30

In [35]:
#import the function
import os
import re
import gradio as gr
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferWindowMemory
from langchain.tools import tool

In [37]:
# LOAD API KEY
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [38]:
# CONFIGURE THE LLM

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=GOOGLE_API_KEY,
    convert_system_message_to_human=True,
)

In [39]:
# BASIC HELPERS

def extract_numbers(text):
    """Extract all numbers from a text string."""
    return [float(x) for x in re.findall(r"-?\d+\.?\d*", text)]

last_tool_used = None

In [41]:
# ADVANCED TOOLS
@tool
def add_numbers(text: str) -> str:
    """Add two numbers extracted from the text."""
    global last_tool_used
    nums = extract_numbers(text)
    if len(nums) < 2:
        return "Please provide two numbers to add."
    last_tool_used = "add_numbers"
    return f"The sum of {int(nums[0])} and {int(nums[1])} is {int(nums[0] + nums[1])}."


@tool
def multiply_numbers(text: str) -> str:
    """Multiply two numbers extracted from the text."""
    global last_tool_used
    nums = extract_numbers(text)
    if len(nums) < 2:
        return "Please provide two numbers to multiply."
    last_tool_used = "multiply_numbers"
    return f"The product of {int(nums[0])} and {int(nums[1])} is {int(nums[0] * nums[1])}."


@tool
def positive_prompt_tool(topic: str) -> str:
    """Generate a positive, creative prompt about a given topic."""
    global last_tool_used
    last_tool_used = "positive_prompt_tool"
    return llm.invoke(
        f"Generate a positive, creative prompt about: {topic}"
    ).content


@tool
def negative_prompt_tool(topic: str) -> str:
    """Generate a negative or adversarial prompt about a given topic."""
    global last_tool_used
    last_tool_used = "negative_prompt_tool"
    return llm.invoke(
        f"Generate a negative or adversarial prompt about: {topic}"
    ).content

@tool
def suicide_related_tool(query: str) -> str:
    """Provide an empathetic, supportive message for self-harm concerns."""
    global last_tool_used
    last_tool_used = "suicide_related_tool"
    return llm.invoke(
        "Provide a compassionate, empathetic message for someone expressing "
        "self-harm thoughts. Avoid instructions. Provide helpful helpline info."
    ).content

In [42]:
# ADDED ALL THE TOOLS
tools = [
    add_numbers,
    multiply_numbers,
    positive_prompt_tool,
    negative_prompt_tool,
    suicide_related_tool,
]

In [43]:
# MEMORY and AGENT

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=100,
    return_messages=True,
)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=False,
)

In [44]:
# CHAT LOGIC

ui_history = []

def chat(user_message):
    global ui_history, last_tool_used

    # Ask which tool was used last
    if "which tool" in user_message.lower():
        response = (
            f"I used the tool '{last_tool_used}' for the previous calculation."
            if last_tool_used else
            "I did not use any tools for the previous question."
        )
    else:
        last_tool_used = None
        response = agent.run(user_message)

    # Styled user bubble
    user_html = f'''
        <div style="background:#e8e8e8; padding:10px; border-radius:8px; margin:6px;">
            <strong>You:</strong> {user_message}
        </div>
    '''

    # Styled bot bubble
    response_html = f'''
        <div style="background:#ffffff; padding:10px; border-radius:8px; margin:6px;">
            <strong>Result:</strong> {response}
        </div>
    '''

    ui_history.append(user_html)
    ui_history.append(response_html)

    return "".join(ui_history)


def clear_ui():
    global ui_history, last_tool_used
    ui_history = []
    last_tool_used = None
    return ""

In [45]:
# GRADIO FRONTEND

css = """
#submit-btn {
    background-color: orange !important;
    color: white !important;
    font-weight: bold;
}

/* Scrollable chat window */
#chat-box {
    max-height: 500px;
    overflow-y: auto;
    border: 1px solid #aaa;
    padding: 10px;
    background: #fafafa;
    border-radius: 12px;
}
"""

with gr.Blocks(css=css) as demo:
    gr.Markdown("# LangChain Agent with Tools\n A LangChain agent that automatically decides which tool to use.")

    user_input = gr.Textbox(lines=2, placeholder="Type your message here...")

    output = gr.HTML(elem_id="chat-box")

    with gr.Row():
        submit_btn = gr.Button("Submit", elem_id="submit-btn")
        clear_btn = gr.Button("Clear")

    submit_btn.click(chat, user_input, output)
    clear_btn.click(clear_ui, None, output)

demo.launch()

/tmp/ipython-input-4213211957.py:21: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://11b1bf0eabb38463ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
